In [13]:
!pip -q install chromadb langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


In [6]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [7]:
!unzip -q new_articles.zip -d new_articles

In [1]:
from google.colab import userdata

In [56]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = userdata.get('HF_TOKEN')
groq_api_key=userdata.get("groq_api")

In [58]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.2 MB/s eta 0:00:00


In [59]:
from langchain_groq import ChatGroq

In [62]:
llm = ChatGroq(groq_api_key=groq_api_key,temperature=0, model_name="mixtral-8x7b-32768")

In [36]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [19]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [22]:
loader=DirectoryLoader("/content/new_articles/",glob="./*.txt",loader_cls=TextLoader)

In [23]:
documents=loader.load()

In [25]:
documents[0]

Document(metadata={'source': '/content/new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt'}, page_content='Partners of 3one4 Capital, a venture capital firm in India, recently went on a road show to raise a new fund. Within two and a half months, at the height of the worsening global economy, they had secured $200 million. It’s the fourth marquee fund for the Bengaluru-headquartered fund, whose portfolio includes four unicorn startups.\n\nThe fund, sixth overall for 3one4 Capital, was oversubscribed to $250 million but the firm is accepting only $200 million to keep itself lean and disciplined, said Pranav Pai, co-founder and partner at 3one4 Capital. The firm’s decision to limit the fund size is emblematic of its strategic choices, which have set it apart from other Indian venture firms.\n\n“We are known to give good returns. Our performance has been benchmarked among the best leading performing funds in the space. So we asked ourselves the hard

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [30]:
text=text_splitter.split_documents(documents)

In [32]:
len(text)

233

In [33]:
text[1]

Document(metadata={'source': '/content/new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt'}, page_content='“We are known to give good returns. Our performance has been benchmarked among the best leading performing funds in the space. So we asked ourselves the hard questions, can we continue our performance with a larger fund size? Do we even need that much capital for the early-stage?” said Pai in an interview with TechCrunch.\n\nIn recent years, a surge of venture capital firms in India have raised unprecedentedly large funds, sparking concerns about the responsible allocation of this capital, particularly for early-stage startups. Critics question whether there are enough viable companies in the Indian market to absorb and effectively utilize such significant investments.')

## ChromaDB

In [37]:
from langchain import embeddings
persist_directory='db'

embeddings=hf

vectordb=Chroma.from_documents(documents=text,
                               embedding=embeddings,
                               persist_directory=persist_directory)

In [38]:
# Saving to the database

vectordb.persist()
vectordb=None

<ipython-input-38-ff320392e935>:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [39]:
# Loading the database from disk
vectordb=Chroma(persist_directory=persist_directory,
                embedding_function=embeddings)

<ipython-input-39-7961b24110b8>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb=Chroma(persist_directory=persist_directory,


# Retriever

In [40]:
retriever=vectordb.as_retriever()

In [42]:
answer=retriever.get_relevant_documents("What is Hugging face")
len(answer)

4

In [43]:
retriever=vectordb.as_retriever(search_kwargs={'k':2})

In [46]:
retriever.search_type

'similarity'

In [47]:
retriever.search_kwargs

{'k': 2}

In [64]:
answer=retriever.get_relevant_documents("How much money microsoft raise")
len(answer)

2

## Chain

In [48]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [63]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [67]:
query="How much money did microsoft raise?"
llm_respond=qa_chain({"query":query})
llm_respond['result']

'Based on the information provided, Microsoft raised around $10 billion. This is mentioned in the first piece of context, which discusses a big investment from Microsoft announced earlier in 2023. The investment was confirmed to be around $10 billion.'

In [68]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')
  for source in llm_response["source_documents"]:
    print(source.metadata['source'])

In [69]:
query="How much money did microsoft raise?"
llm_respond=qa_chain({"query":query})
process_llm_response(llm_respond)

Based on the information provided, Microsoft raised around $10 billion. This is mentioned in the first piece of context, which discusses a big investment from Microsoft announced earlier in 2023. The investment was confirmed to be around $10 billion.


Sources:
/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/content/new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt


In [70]:
query="What is hugging face"
llm_respond=qa_chain({"query":query})
process_llm_response(llm_respond)

Hugging Face is a platform for hosting and running AI models. Spawning is partnering with Hugging Face to add a new info box on Hugging Face that will alert users to the proportion of "opted-out" data within text-to-image datasets. This box will also link to a Spawning API sign-up page so that model trainers can remove opted-out images at training time. Hugging Face is also a part of the BigCode project, which aims to develop state-of-the-art AI systems for code in an open and responsible way. They supplied an in-house compute cluster of 512 Nvidia V100 GPUs to train the StarCoder model.


Sources:
/content/new_articles/05-03-spawning-lays-out-its-plans-for-letting-creators-opt-out-of-generative-ai-training.txt
/content/new_articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt


# Deleting the Database

In [ ]:
!zip -r db.zip ./db

In [ ]:
vectordb.delete_collection()
vectordb.persist()

# Deleting the directory

!rm -rf db/

# Unzipping the database

In [ ]:
!unzip db.zip